<a href="https://colab.research.google.com/github/erofale/encoderProject/blob/master/Code/autoencoder_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
import keras.backend as K
from keras.layers import Lambda
from keras.regularizers import L1L2
import numpy as np
import random
import pandas as pd
from sklearn.metrics import mean_squared_error

In [ ]:
def generator_test(func, file_name, n = 10000):
  arr = []
  for i in range(n):
    x = random.uniform(0., 10000.)
    arr.append([x, 5. - x, 0, 0])
  df = pd.DataFrame(arr, columns=['x1', 'x2', 'x3', 'x4'])
  df.to_csv('/content/sample_data/' + file_name)

In [ ]:
def func(x):
  return x[0] + x[1]

generator_test(func, 'lin.csv')
data = pd.read_csv('/content/sample_data/lin.csv', index_col=0).to_numpy('float32')

In [ ]:
def normire(data, entry_min, entry_max):
  entry_range = entry_max - entry_min
  norm_data = [0.0 + (data[i] - entry_min) / entry_range for i in range(len(data))]
  return norm_data

def renormire(norm_data, entry_min, entry_max):
  entry_range = entry_max - entry_min
  data = [entry_min + (norm_data[i] - 0.0) * entry_range for i in range(len(norm_data))]
  return data

In [ ]:
class AutoencoderClass():
  def __init__(self, func, input_dim : int, encoding_dim : int, activations : list, enc_type : str):
    self.func = func
    self.input_dim = input_dim
    self.encoding_dim = encoding_dim  # Размерность кодированного представления
    self.activations = activations
    #self.loss = loss
    self.enc_type = enc_type
    self.aes_types = {'dense': self.__create_dense_ae, 'deep':self.__create_deep_dense_ae, 'conv':self.__create_deep_conv_ae}
    try:
      self.autoencoder = self.aes_types[self.enc_type]()
      self.autoencoder.compile(optimizer = 'adam', loss = self.custom_loss)
    except KeyError as e:
        # можно также присвоить значение по умолчанию вместо бросания исключения
        raise ValueError('Undefined unit: {}'.format(e.args[0]))

  def fit(self, train_data, test_data, epochs : int, batch_size : int, shuffle : bool):
    if self.enc_type != 'conv':
      self.autoencoder.fit(train_data, train_data,
                  epochs=epochs,
                  batch_size=batch_size,
                  shuffle=shuffle,
                  validation_data=(test_data, test_data))
    
    else:
      grid_train = []
      grid_test = []
      for i in range(len(train_data)):
        xx, yy = np.meshgrid(train_data[i], train_data[i])
        grid_train.append(xx)

      for i in range(len(test_data)):
        xx, yy = np.meshgrid(test_data[i], test_data[i])
        grid_test.append(xx)
      
      self.autoencoder.fit(grid_train, grid_train,
            epochs=epochs,
            batch_size=batch_size,
            shuffle=shuffle,
            validation_data=(grid_test, grid_test))

  def predict(self, x_vector):
    if self.enc_type != 'conv':
      return self.autoencoder.predict(x_vector)
    else:
      return self.autoencoder.predict(x_vector)[0]


  def get_aec_type(self):
    return self.enc_type

  def custom_loss(self, x_true, x_pred):
    return K.mean(K.square(self.func(x_pred) - self.func(x_true)))

  # Сжимающий автоэнкодер
  def __create_dense_ae(self):
    # Энкодер
    # Входной плейсхолдер
    input_data = Input(shape=(self.input_dim,))
    # Кодированное полносвязным слоем представление
    encoded = Dense(self.encoding_dim, activation = self.activations[0])(input_data)
    
    # Декодер
    # Раскодированное другим полносвязным слоем изображение
    input_encoded = Input(shape = (self.encoding_dim,))
    decoded = Dense(self.input_dim, activation = self.activations[1])(input_encoded)

    # Модели, в конструктор первым аргументом передаются входные слои, а вторым выходные слои
    # Другие модели можно так же использовать как и слои
    encoder = Model(input_data, encoded, name = "encoder")
    decoder = Model(input_encoded, decoded, name = "decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name = "autoencoder")
    return autoencoder

  # Глубокий автоэнкодер
  def __create_deep_dense_ae(self):
    # Энкодер
    input_data = Input(shape=(self.input_dim,))
    ############# Здесь фиксить надо с размерностями слоёв и как подавать активации на вход##########
    x = Dense(self.encoding_dim*3, activation='relu')(input_data)
    x = Dense(self.encoding_dim*2, activation='relu')(x)
    encoded = Dense(self.encoding_dim, activation='linear')(x)
    
    # Декодер
    input_encoded = Input(shape=(self.encoding_dim,))
    ############# Здесь фиксить надо с размерностями слоёв ##########
    x = Dense(self.encoding_dim*2, activation='relu')(input_encoded)
    x = Dense(self.encoding_dim*3, activation='relu')(x)
    decoded = Dense(self.input_dim, activation='sigmoid')(x)
    
    # Модели
    encoder = Model(input_data, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name="autoencoder")
    return autoencoder

  # Сверточный автоэнкодер
  def __create_deep_conv_ae(self):
    input_data = Input(shape=(self.input_dim, self.input_dim, 1))

    x = Conv2D(25, (2, 2), activation='relu', padding='same')(input_data)
    x = MaxPooling2D((2, 2), padding='same')(x)
    #x = Conv2D(32, (2, 2), activation='relu', padding='same')(x)
    #x = MaxPooling2D((2, 2), padding='same')(x)
    encoded = Conv2D(1, (2, 2), activation='relu', padding='same')(x)

    # На этом моменте представление  (7, 7, 1) т.е. 49-размерное

    input_encoded = Input(shape=(7, 7, 1))
    #x = Conv2D(32, (7, 7), activation='relu', padding='same')(input_encoded)
    #x = UpSampling2D((2, 2))(x)
    x = Conv2D(25, (2, 2), activation='relu', padding='same')(input_encoded)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (2, 2), activation='sigmoid', padding='same')(x)

    # Модели
    encoder = Model(input_data, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name="autoencoder")
    return autoencoder

In [ ]:
data = pd.read_csv('/content/sample_data/lin.csv', index_col=0).to_numpy('float32')
entry_min = -10000.
entry_max = 10000.
norm_data = normire(data, entry_min, entry_max)
data_train = np.array(norm_data[0:8000])
data_test = np.array(norm_data[8000:10000])

autoencoder = AutoencoderClass(func, 4, 2, list(['relu', 'sigmoid']), 'dense')

In [ ]:
autoencoder.fit(data_train, data_test, 10, 256, True)

In [ ]:
data_predict = []
for i in range(10):
  print(f'Orig: {data[i]}, Y: {data[i][0] + data[i][1]}')
  pred = autoencoder.predict(data[i].reshape(1,3))
  data_predict.append(renormire(pred[0].reshape(3,), -10000., 10000.))
  print(f'Pred: {data_predict[i]}, Y: {data_predict[i][0] + data_predict[i][1]}\n')

print(mean_squared_error(data[0:10], data_predict))